In [1]:
import pandas as pd
import warnings

path = r'/home/dwd/proj/Diversity-Improvement-in-CBR/CleanedDATA V12-05-2021.csv'
df = pd.read_csv(path, sep=';', encoding='windows-1252')

warnings.filterwarnings('ignore')

series = df['Publication identifier,,,,,,,,,,,,,,,,,,']
for i in range(len(series)):
    j_comma = series[i].find(',')
    if j_comma > 0:
        series[i] = series[i][:j_comma]
    
df.rename(columns={'Publication identifier,,,,,,,,,,,,,,,,,,': 'Publication identifier'}, inplace=True)
# df.to_csv(path, sep=';', index=False, encoding='windows-1252')

In [2]:
from utils.case import Query, Case, Description, GC, Solution
from utils.casebase import CaseBase, MCNN_CaseBase
from utils.utils import retrieve_topk

# Build the case base from the dataframe
attr_names = ['Task', 'Case study type', 'Case study', 'Online/Off-line', 'Input for the model',
                    'Model Approach', 'Model Type', 'Models', 'Data Pre-processing', 'Complementary notes', 'Publication identifier',
                    'Performance indicator', 'Performance', 'Publication Year']
df = df[attr_names]
cb = CaseBase.from_dataframe(df)

# Build the query
df_qry = df[attr_names[:5]]
query = Query.from_series(df_qry.iloc[0])

# Retrieve the top-5 most similar cases to the query.
retrieve_topk(query, cb, weights=[1, 1, 1, 1, 1], k=5)


[(Case 0, 1.0),
 (Case 11, 1.0),
 (Case 1, 0.9800000000000001),
 (Case 47, 0.9800000000000001),
 (Case 48, 0.9800000000000001)]

In [5]:
# Apply CBM method - MCNN - to organize the case base.
# Initialize the MCNN Case Base
mcnn_cb = MCNN_CaseBase(cb.cases, desc_attrs=attr_names[:5], sol_attrs=attr_names[5:], 
                        thr_desc=0.7, thr_sol=0.7, _seed=42)

print(len(mcnn_cb.descriptions), len(set(mcnn_cb.descriptions)))
print(len(mcnn_cb.solutions), len(set(mcnn_cb.solutions)))

12 12
34 34


### TODO: test & evaluate (similarity, diversity, etc.)